In [1]:
from fastai.vision.all import *

/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
#confirm that Python 3.10 is running
!which python
!python --version

/Users/bruceantley/anaconda3/envs/fastai310/bin/python
Python 3.10.18


In [3]:
from pathlib import Path

base_path = Path('/Users/bruceantley/Documents/face_classifier_310/fine_tune_sets/round2')
model_path = Path('/Users/bruceantley/Documents/face_classifier_310/model_exports/round1_final.pkl')

In [4]:
dls = ImageDataLoaders.from_folder(
    base_path,
    train='.',                  # real and fake are both directly inside round1/
    valid_pct=0.2,              # 20% of each class held out for validation
    item_tfms=Resize(256),      # Resize all to 256x256
    batch_tfms=[                # Data augmentation + normalization
        *aug_transforms(),
        Normalize.from_stats(*imagenet_stats)
    ],
    bs=16,
    seed=42
)

In [5]:
learn = cnn_learner(dls, resnet50)
learn = load_learner(model_path)
learn.dls = dls  # override the data loaders with your fine-tune set

/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [6]:
learn.fine_tune(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.878137,0.710610,0.686869,00:31


epoch,train_loss,valid_loss,accuracy,time
0,0.553003,0.557245,0.727273,00:26
1,0.440050,0.479105,0.828283,00:26
2,0.376810,0.470782,0.808081,00:26


In [7]:
# First run 1 epoch with frozen layers
learn.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.239299,0.568763,0.787879,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.240760,0.487781,0.828283,00:26


In [9]:
# Then unfreeze and run 9 more
learn.unfreeze()
learn.fit_one_cycle(9, lr_max=slice(1e-6, 1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.154838,0.501702,0.818182,00:27
1,0.158631,0.492981,0.818182,00:27
2,0.133926,0.472623,0.838384,00:27
3,0.132331,0.454203,0.838384,00:27
4,0.123233,0.434042,0.828283,00:27
5,0.145475,0.462228,0.828283,00:26
6,0.122732,0.482054,0.828283,00:26
7,0.119235,0.474618,0.828283,00:26
8,0.118683,0.491670,0.818182,00:26


In [15]:
learn.export('/Users/bruceantley/Documents/face_classifier_310/model_exports/round2_final.pkl')

In [10]:
from fastai.callback.mixup import MixUp
learn = vision_learner(dls, resnet50, metrics=accuracy, cbs=MixUp(0.2))
learn.fine_tune(3)

epoch,train_loss,valid_loss,accuracy,time
0,1.235043,0.762784,0.686869,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.873583,0.913473,0.696970,00:26
1,0.846385,0.764211,0.777778,00:26
2,0.755969,0.813531,0.727273,00:26


In [11]:
learn.fit_one_cycle(5, lr_max=slice(1e-6, 3e-5))

epoch,train_loss,valid_loss,accuracy,time
0,0.540744,0.743035,0.707071,00:27
1,0.630153,0.759302,0.737374,00:26
2,0.601946,0.711018,0.707071,00:26
3,0.606395,0.798078,0.717172,00:26
4,0.603720,0.771600,0.717172,00:26


In [12]:
# 🧪 Predict on a new image
#fake gym
img = PILImage.create('/Users/bruceantley/Desktop/Screenshot 2025-07-06 at 3.38.45 PM.png')  # replace with an actual file
pred, pred_idx, probs = learn.predict(img)
print(f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}")

Prediction: fake; Probability: 0.9950


In [13]:
# 🧪 Predict on a new image
#ariel
img = PILImage.create('/Users/bruceantley/Desktop/Screenshot 2025-07-06 at 3.35.04 PM.png')  # replace with an actual file
pred, pred_idx, probs = learn.predict(img)
print(f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}")

Prediction: real; Probability: 0.9393


In [16]:
# 🧪 Predict on a new image
#fake guy
img = PILImage.create('/Users/bruceantley/Desktop/Screenshot 2025-07-11 at 8.00.30 AM.png')  # replace with an actual file
pred, pred_idx, probs = learn.predict(img)
print(f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}")

Prediction: real; Probability: 0.9789
